# 參數輸入
* 記得covariance matrix 項中要乘上T
* array col合併 np.hstack

In [1]:
import numpy as np
s0 = np.array([100,90,80,110,120]) #vector
K = 100
numsim = 10000
numrep = 20
T = 1
r = 0.5
n = 5   #股票的種類
q = np.array([0.4,0.35,0.3,0.2,0.38])
sigma =np.array( [0.05,0.1,0.23,0.2,0.3] )#vector
corr= [] #matrix

corr=np.array([
    [1, 0.1, 0.01,-0.03,0.04],
    [0, 1, 0.04,0.03,-0.05],
    [0, 0, 1, 0.05 , -0.07],
    [0, 0, 0 ,1 , 0.09],
    [0, 0, 0, 0 , 1]
])
corr = corr.T+corr-np.identity(5)
#抓column一排很麻煩 要先轉置在抓
covmat = (np.transpose([sigma]).dot([sigma])) * corr *T
covmat

array([[ 0.0025  ,  0.0005  ,  0.000115, -0.0003  ,  0.0006  ],
       [ 0.0005  ,  0.01    ,  0.00092 ,  0.0006  , -0.0015  ],
       [ 0.000115,  0.00092 ,  0.0529  ,  0.0023  , -0.00483 ],
       [-0.0003  ,  0.0006  ,  0.0023  ,  0.04    ,  0.0054  ],
       [ 0.0006  , -0.0015  , -0.00483 ,  0.0054  ,  0.09    ]])

In [2]:
s0 = np.array([95]*2) #vector
K = 100
numsim = 10000
numrep = 20
T = 0.5
r = 0.1
n = 2   #股票的種類
q = np.array([0.05]*2)
sigma =np.array( [0.5]*2 )#vector
corr= [] #matrix

corr=np.array([
    [1,1],
    [0,1]
])
corr = corr.T+corr-np.identity(n)
#抓column一排很麻煩 要先轉置在抓
covmat = (np.transpose([sigma]).dot([sigma])) * corr *T
covmat

array([[0.125, 0.125],
       [0.125, 0.125]])

In [3]:
s0 = np.array([95]*2) #vector
K = 100
numsim = 10000
numrep = 20
T = 0.5
r = 0.1
n = 2   #股票的種類
q = np.array([0.05]*2)
sigma =np.array( [0.5]*2 )#vector
corr= [] #matrix

corr=np.array([
    [1,-1],
    [0,1]
])
corr = corr.T+corr-np.identity(n)
#抓column一排很麻煩 要先轉置在抓
covmat = (np.transpose([sigma]).dot([sigma])) * corr *T
covmat

array([[ 0.125, -0.125],
       [-0.125,  0.125]])

In [4]:
import numpy as np
s0 = np.array([95]*5) #vector
K = 100
numsim = 10000
numrep = 20
T = 0.5
r = 0.1
n = 5   #股票的種類
q = np.array([0.05]*5)
sigma =np.array( [0.5]*5 )#vector
corr= [] #matrix

corr=np.array([
    [1, 0.5, 0.5,0.5,0.5],
    [0, 1, 0.5,0.5,0.5],
    [0, 0, 1, 0.5 , 0.5],
    [0, 0, 0 ,1 , 0.5],
    [0, 0, 0, 0 , 1]
])
corr = corr.T+corr-np.identity(n)
#抓column一排很麻煩 要先轉置在抓
covmat = (np.transpose([sigma]).dot([sigma])) * corr *T
covmat

array([[0.125 , 0.0625, 0.0625, 0.0625, 0.0625],
       [0.0625, 0.125 , 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.125 , 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, 0.125 , 0.0625],
       [0.0625, 0.0625, 0.0625, 0.0625, 0.125 ]])

# Cholesky decomposition:
* $C=A^T * A$

In [5]:
def choldecomp(covmat):
    n = len(covmat)
    A = np.eye(n, dtype = 'float')
    for Arow in range(n+1):
        for Acol in range(n+1):
            if Arow ==1:
                if Acol == Arow:
                    A[Arow-1][Arow-1] = np.sqrt(covmat[Arow-1][Arow-1])
                else:
                    A[Arow-1][Acol-1] = covmat[Arow-1][Acol-1] / A[Arow-1][Arow-1]
            else:
                if Acol == Arow:
                    res = sum(A.T[Arow-1][range(Arow-1)]**2)
                    A[Arow-1][Arow-1] = np.sqrt( covmat[Arow-1][Arow-1]-res )
                else:
                    res = sum(A.T[Arow-1][range(Arow-1)]*A.T[Acol-1][range(Arow-1)])
                    A[Arow-1][Acol-1] = (covmat[Arow-1][Acol-1]-res) / A[Arow-1][Arow-1]
    return A
cholmat = choldecomp(covmat)


# Rainbow option計算函數

# 公式:
$ 
[Z_1,Z_2,...]*A = Z*A = [r1,r2,...] \quad 再加上+[\mu_1,\mu_2,...]  \\
e^{上式} = [S_{1T},S_{2T},...]
$

In [6]:
def M2rainbowpricecount(sample,cholmat):
    logmu = np.log(s0)+(r-q-sigma**2 /2)*T
    StMCsample = sample.dot(cholmat) + np.ones([numsim,n]).dot(np.diag(logmu)) 
    StMCsample = np.exp(StMCsample)
    rainbowprice = [[]]*numsim
    for i in range(len(StMCsample)):
        rainbowprice[i] = max( max(StMCsample[i])-K , 0 )*np.exp(-r*T)
    return np.mean(rainbowprice)

In [7]:
np.exp(-r*T)

0.951229424500714

# Bonus1
* 取一半z1,-z1

In [8]:
def antisamplegen(Z01normsample):
    antivanormsample = Z01normsample[:][[range(int(numsim/2))]]
    antivanormsample = np.concatenate((antivanormsample,-antivanormsample),axis=0)
    antivamn = np.mean(antivanormsample)
    antivastd = np.std(antivanormsample)
    antivanormsample = (antivanormsample-antivamn)/antivastd
    return antivanormsample

# Bonus2
* 先將抽樣模擬成真正的無相關性

In [9]:
def invcholgen(Z01normsample):
    antivanormsample = Z01normsample[:][[range(int(numsim/2))]]
    antivanormsample = np.concatenate((antivanormsample,-antivanormsample),axis=0)
    A = choldecomp(np.cov(antivanormsample.T))
    A_inv = np.linalg.inv(A)
    invcholsample = antivanormsample.dot(A_inv)
    return invcholsample

# 所有抽樣一起做

In [10]:
rainbowprice = []
bonus1 = []
bonus2 = []
for i in range(numrep):
    Z01normsample = np.random.normal(0,1,numsim*n).reshape(numsim,n)
    rainbowprice.append( M2rainbowpricecount(Z01normsample,cholmat) )
    bonus1.append( M2rainbowpricecount(antisamplegen(Z01normsample),cholmat) )
    bonus2.append( M2rainbowpricecount(invcholgen(Z01normsample),cholmat) )
pricemn = np.mean(rainbowprice)
pricestd = np.std(rainbowprice) 
print("基本抽樣: %f" %pricemn)
print("Rainbow option價格95%%信賴區間為=(%f,%f)" %(pricemn-2*pricestd,pricemn+2*pricestd))
rainbowprice = bonus1
pricemn = np.mean(rainbowprice)
pricestd = np.std(rainbowprice) 
print("bonus1: %f" %pricemn)
print("Rainbow option價格95%%信賴區間為=(%f,%f)" %(pricemn-2*pricestd,pricemn+2*pricestd))
rainbowprice = bonus2
pricemn = np.mean(rainbowprice)
pricestd = np.std(rainbowprice) 
print("bonus2: %f" %pricemn)
print("Rainbow option價格95%%信賴區間為=(%f,%f)" %(pricemn-2*pricestd,pricemn+2*pricestd))

基本抽樣: 30.260046
Rainbow option價格95%信賴區間為=(29.713699,30.806394)
bonus1: 30.380134
Rainbow option價格95%信賴區間為=(30.259330,30.500937)
bonus2: 30.390381
Rainbow option價格95%信賴區間為=(30.281810,30.498953)
